In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import matplotlib.patches as patches

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.datasets.mnist as mnist
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors

import torch
from torch import nn
from torch import optim

import cv2
import os
from PIL import Image


import torch.optim as optim

In [31]:
plt.rcParams['image.cmap'] = 'gray'

In [32]:
mnist.MNIST('.', download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: .
    Split: Train

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [34]:
train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
dataset1 = mnist.MNIST('.', train=True, transform=T.ToTensor())
dataset2 = mnist.MNIST('.', train=False, transform=T.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)



In [35]:

model = Net()
optimizer = optim.Adadelta(model.parameters())

for epoch in range(1,5):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301912
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.357258
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.104665
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.252001
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.117704
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.233640
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.036674
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.201668
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.190179
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.175451

Test set: Average loss: 0.0568, Accuracy: 9829/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.043467
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.097622
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.028796
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.135394
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.040372
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.126412
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.085305
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.097112
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.120595
T

In [36]:
torch.save(model.state_dict(), "mnist_cnn.pt")

In [50]:
dataset2[42][0][0].shape

torch.Size([28, 28])

In [56]:
test_loader

In [61]:
img = T.ToPILImage()(dataset2[54][0])
img

In [66]:
model.eval()
reaction = model(T.ToTensor()(img).unsqueeze(0))
reaction

tensor([[-1.6637e+01, -2.3055e+01, -2.7066e+01, -2.7932e+01, -2.1231e+01,
         -1.0692e+01, -2.2888e-05, -3.1402e+01, -1.6947e+01, -2.5722e+01]],
       grad_fn=<LogSoftmaxBackward0>)

In [67]:
reaction.argmax()

tensor(6)

Маємо 6, як і має бути